### Fitting models and materials database
- Models: Drude, Lorentz, Critical-points poles
$$ -\frac{ω_p^2}{ω^2+iγ_pω},\quad
    \frac{2f_pω_p^2}{ω_p^2-(ω+iγ_p)^2}, \quad
    \frac{2f_pω_p^2-2ig_pω_p(ω+iγ_p)}{ω_p^2-(ω+iγ_p)^2} $$
- Materials: [($ϵ_1$, $n_1$, $k_1$), ($ϵ_2$, $n_2$, $k_2$), ...]

In [30]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

def nk(ω, model, param):
    def Dr(ω, p):
        ωp, γp = p
        return - ωp**2 / (ω**2+1j*γp*ω)
    
    def Lo(ω, p):
        ωp, γp, fp = p
        return 2 * fp*ωp**2 / (ωp**2 - (ω+1j*γp)**2)
    
    def CP(ω, p):
        ωp, γp, fp, gp = p
        return 2 * (fp*ωp**2 - 1j*gp*ωp*(ω+1j*γp)) / (ωp**2 - (ω+1j*γp)**2)
    
    ϵ, i = param[0], 1
    for m in model:
        ϵ += {2:Dr, 3:Lo, 4:CP}[m](ω, param[i:i+m])
        i += m
    return ϵ**0.5

def plotFit(data, model, param, λ0):
    vnk = np.vectorize(lambda ω: nk(ω, model, param))
    ω = np.linspace(data[0,0], data[-1,0], 200)
    n = vnk(ω)
    
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(λ0/ω if λ0 else ω, n.real, label='n (fit)')
    ax.plot(λ0/ω if λ0 else ω, n.imag, label='k (fit)')
    ax.scatter(λ0/data[:,0] if λ0 else data[:,0], data[:,1], s=3, label='n (table)')
    ax.scatter(λ0/data[:,0] if λ0 else data[:,0], data[:,2], s=3, label='k (table)')
    ax.grid()
    plt.show()

def findFit(material, region, model, poles=[], plot=True):
    λ0 = 1240
    i = max(0, np.argmin(MaterialsDB[material][:,0] < λ0/region[1])-1)
    j = np.argmin(MaterialsDB[material][:,0] < λ0/region[0])
    data = MaterialsDB[material][i:j+1]
    
    x0 = [1.0]
    bounds = [(1.0, None)]
    for m in model:
        x0 += [poles[0] if len(poles) else 1.0, 0.1, 1.0, 0.1][:m]
        bounds += [(0.0, None), (0.0, None), (None, None), (None, None)][:m]
        poles = poles[1:]
            
    def f(p):
        vnk = np.vectorize(lambda ω: nk(ω, model, p))
        return sum(abs(vnk(data[:,0])-(data[:,1]+1j*data[:,2]))**2)
        
    fit = opt.minimize(f, x0, bounds=bounds)
    p = []
    x = fit.x[1:]
    for m in model:
        p.append('{' + ','.join(str(round(i, 4)) for i in x[:m]) + '}')
        x = x[m:]
        
    if plot:
        print('Success=', fit.success, ', σ=', round((fit.fun/len(data))**0.5, 3), sep='')
        plotFit(data, model, fit.x, λ0)
    print(f'matter {material} = {{{{{round(fit.x[0], 4)}}},{{' + ','.join(p) + '}};')

In [31]:
MaterialsDB = {}

# Johnson and Christy, Phys. Rev. B 6, 4370 (1972)
MaterialsDB['AgJC']=np.array([(0.64,0.24,14.08),(0.77,0.15,11.85),(0.89,0.13,10.1),(1.02,0.09,8.828),(1.14,0.04,7.795),(1.26,0.04,6.992),(1.39,0.04,6.312),(1.51,0.04,5.727),(1.64,0.03,5.242),(1.76,0.04,4.838),(1.88,0.05,4.483),(2.01,0.06,4.152),(2.13,0.05,3.858),(2.26,0.06,3.586),(2.38,0.05,3.324),(2.5,0.05,3.093),(2.63,0.05,2.869),(2.75,0.04,2.657),(2.88,0.04,2.462),(3,0.05,2.275),(3.12,0.05,2.07),(3.25,0.05,1.864),(3.37,0.07,1.657),(3.5,0.1,1.419),(3.62,0.14,1.142),(3.74,0.17,0.829),(3.87,0.81,0.392),(3.99,1.13,0.616),(4.12,1.34,0.964),(4.24,1.39,1.161),(4.36,1.41,1.264),(4.49,1.41,1.331),(4.61,1.38,1.372),(4.74,1.35,1.387),(4.86,1.33,1.393),(4.98,1.31,1.389)])
MaterialsDB['AuJC']=np.array([(0.64,0.92,13.78),(0.77,0.56,11.21),(0.89,0.43,9.519),(1.02,0.35,8.145),(1.14,0.27,7.15),(1.26,0.22,6.35),(1.39,0.17,5.663),(1.51,0.16,5.083),(1.64,0.14,4.542),(1.76,0.13,4.103),(1.88,0.14,3.697),(2.01,0.21,3.272),(2.13,0.29,2.863),(2.26,0.43,2.455),(2.38,0.62,2.081),(2.5,1.04,1.833),(2.63,1.31,1.849),(2.75,1.38,1.914),(2.88,1.45,1.948),(3,1.46,1.958),(3.12,1.47,1.952),(3.25,1.46,1.933),(3.37,1.48,1.895),(3.5,1.5,1.866),(3.62,1.48,1.871),(3.74,1.48,1.883),(3.87,1.54,1.898),(3.99,1.53,1.893),(4.12,1.53,1.889),(4.24,1.49,1.878),(4.36,1.47,1.869),(4.49,1.43,1.847),(4.61,1.38,1.803),(4.74,1.35,1.749),(4.86,1.33,1.688),(4.98,1.33,1.631)])
MaterialsDB['CuJC']=np.array([(0.64,1.09,13.43),(0.77,0.76,11.12),(0.89,0.6,9.439),(1.02,0.48,8.245),(1.14,0.36,7.217),(1.26,0.32,6.421),(1.39,0.3,5.768),(1.51,0.26,5.18),(1.64,0.24,4.665),(1.76,0.21,4.205),(1.88,0.22,3.747),(2.01,0.3,3.205),(2.13,0.7,2.704),(2.26,1.02,2.577),(2.38,1.18,2.608),(2.5,1.22,2.564),(2.63,1.25,2.483),(2.75,1.24,2.397),(2.88,1.25,2.305),(3,1.28,2.207),(3.12,1.32,2.116),(3.25,1.33,2.045),(3.37,1.36,1.975),(3.5,1.37,1.916),(3.62,1.36,1.864),(3.74,1.34,1.821),(3.87,1.38,1.783),(3.99,1.38,1.729),(4.12,1.4,1.679),(4.24,1.42,1.633),(4.36,1.45,1.633),(4.49,1.46,1.646),(4.61,1.45,1.668),(4.74,1.41,1.691),(4.86,1.41,1.741),(4.98,1.37,1.783)])

# Handbook of Chemistry & Physics (2006)
MaterialsDB['Ag']=np.array([(0.5,0.67,18.32),(1.0,0.28,9.03),(1.5,0.27,5.79),(2.0,0.27,4.18),(2.5,0.24,3.09),(3.0,0.23,2.27),(3.25,0.23,1.86),(3.5,0.21,1.42),(3.6,0.23,1.13),(3.7,0.3,0.77),(3.77,0.53,0.40),(3.8,0.73,0.30),(3.9,1.3,0.36),(4.0,1.61,0.60),(4.1,1.73,0.85),(4.2,1.75,1.06),(4.3,1.73,1.13),(4.5,1.69,1.28),(4.75,1.61,1.34),(5.0,1.55,1.36)])
MaterialsDB['Au']=np.array([(0.5,0.39,16.61),(0.6,0.28,13.78),(0.7,0.22,11.75),(0.8,0.18,10.21),(0.9,0.15,9.01),(1.0,0.13,8.03),(1.2,0.1,6.54),(1.4,0.08,5.44),(1.6,0.08,4.56),(1.8,0.09,3.82),(2.0,0.13,3.16),(2.1,0.18,2.84),(2.2,0.24,2.54),(2.4,0.5,1.86),(2.5,0.82,1.59),(2.6,1.24,1.54),(2.7,1.43,1.72),(2.8,1.46,1.77),(2.9,1.5,1.79),(3.0,1.54,1.80),(3.1,1.54,1.81),(3.2,1.54,1.80),(3.3,1.55,1.78),(3.4,1.56,1.76),(3.5,1.58,1.73),(3.6,1.62,1.73),(3.7,1.64,1.75),(3.8,1.63,1.79),(3.9,1.59,1.81),(4.0,1.55,1.81),(4.1,1.51,1.79),(4.2,1.48,1.78),(4.3,1.45,1.77),(4.4,1.41,1.76),(4.5,1.35,1.74),(4.6,1.3,1.69),(4.7,1.27,1.64),(4.8,1.25,1.59),(4.9,1.23,1.54),(5.0,1.22,1.49),(5.2,1.21,1.40),(5.4,1.21,1.33),(5.6,1.21,1.27),(5.8,1.21,1.20),(6.0,1.22,1.14),(6.2,1.24,1.09),(6.4,1.25,1.05),(6.6,1.27,1.01),(6.8,1.3,0.97),(7.0,1.34,0.95)])
MaterialsDB['Cu']=np.array([(0.5,1.71,17.63),(1.0,0.44,8.48),(1.5,0.26,5.26),(1.7,0.22,4.43),(1.75,0.21,4.25),(1.8,0.21,4.04),(1.85,0.22,3.85),(1.9,0.21,3.67),(2.0,0.27,3.24),(2.1,0.47,2.81),(2.2,0.83,2.60),(2.3,1.04,2.59),(2.4,1.12,2.60),(2.6,1.15,2.50),(2.8,1.17,2.36),(3.0,1.18,2.21),(3.2,1.23,2.07),(3.4,1.27,1.95),(3.6,1.31,1.87),(3.8,1.34,1.81),(4.0,1.34,1.72),(4.2,1.42,1.64),(4.4,1.49,1.64),(4.6,1.52,1.67),(4.8,1.53,1.71),(5.0,1.47,1.78),(5.2,1.38,1.80),(5.4,1.28,1.78),(5.6,1.18,1.74),(5.8,1.1,1.67),(6.0,1.04,1.59),(6.5,0.96,1.37),(7.0,0.97,1.20)])
MaterialsDB['Al']=np.array([(0.6,2.273,21.403),(0.7,1.77,18.328),(0.8,1.444,15.955),(0.9,1.264,14.021),(1.,1.212,12.464),(1.1,1.201,11.181),(1.2,1.26,10.010),(1.3,1.468,8.949),(1.4,2.237,8.212),(1.5,2.745,8.309),(1.6,2.625,8.597),(1.7,2.143,8.573),(1.8,1.741,8.205),(1.9,1.488,7.821),(2.,1.304,7.479),(2.2,1.018,6.846),(2.4,0.826,6.283),(2.6,0.695,5.800),(2.8,0.598,5.385),(3.,0.523,5.024),(3.2,0.46,4.708),(3.4,0.407,4.426),(3.6,0.363,4.174),(3.8,0.326,3.946),(4.,0.294,3.740),(4.2,0.267,3.552),(4.4,0.244,3.380),(4.6,0.223,3.222),(4.8,0.205,3.076),(5.,0.19,2.942)])
MaterialsDB['Cr']=np.array([(0.54,3.92,7.06),(0.66,3.96,5.95),(0.78,4.13,5.03),(0.9,4.43,4.60),(1.,4.47,4.43),(1.12,4.53,4.31),(1.24,4.5,4.28),(1.36,4.42,4.30),(1.46,4.31,4.32),(1.77,3.84,4.37),(2.,3.48,4.36),(2.2,3.18,4.41),(2.4,2.75,4.46),(2.6,2.22,4.36),(2.8,1.8,4.06),(3.,1.54,3.71),(3.2,1.44,3.40),(3.4,1.39,3.24),(3.6,1.26,3.12),(3.8,1.12,2.95),(4.,1.02,2.76),(4.2,0.94,2.58),(4.4,0.9,2.42),(4.5,0.89,2.35),(4.6,0.88,2.28),(4.7,0.86,2.21),(4.8,0.86,2.13),(4.9,0.86,2.07),(5.,0.85,2.01)])
MaterialsDB['Ni']=np.array([(0.6,3.84,8.35),(0.7,3.59,7.48),(0.8,3.38,6.82),(0.9,3.18,6.23),(1.,3.06,5.74),(1.1,2.97,5.38),(1.2,2.85,5.10),(1.3,2.74,4.85),(1.4,2.65,4.63),(1.5,2.53,4.47),(1.6,2.43,4.31),(1.7,2.28,4.18),(1.8,2.14,4.01),(1.9,2.02,3.82),(2.,1.92,3.65),(2.1,1.85,3.48),(2.2,1.8,3.33),(2.3,1.75,3.19),(2.4,1.71,3.06),(2.5,1.67,2.93),(2.6,1.65,2.81),(2.7,1.64,2.71),(2.8,1.63,2.61),(2.9,1.62,2.52),(3.,1.61,2.44),(3.1,1.61,2.36),(3.2,1.61,2.30),(3.3,1.61,2.23),(3.4,1.62,2.17),(3.5,1.63,2.11),(3.6,1.64,2.07),(3.7,1.66,2.02),(3.8,1.69,1.99),(3.9,1.72,1.98),(4.,1.73,1.98),(4.2,1.74,2.01),(4.4,1.71,2.06),(4.6,1.63,2.09),(4.8,1.53,2.11),(5.,1.4,2.10)])
MaterialsDB['Pd']=np.array([(0.6,3.8,9.96),(0.72,3.51,8.70),(0.8,3.35,8.06),(1.0,2.99,6.89),(1.1,2.81,6.46),(1.2,2.65,6.10),(1.3,2.5,5.78),(1.4,2.34,5.50),(1.5,2.17,5.22),(1.6,2.08,4.95),(1.7,2.0,4.72),(1.8,1.92,4.54),(1.9,1.82,4.35),(2.0,1.75,4.18),(2.1,1.67,4.03),(2.2,1.6,3.88),(2.3,1.53,3.75),(2.4,1.47,3.61),(2.5,1.41,3.48),(2.6,1.37,3.36),(2.7,1.32,3.25),(2.8,1.29,3.13),(2.9,1.26,3.03),(3.0,1.23,2.94),(3.1,1.2,2.85),(3.2,1.17,2.77),(3.3,1.14,2.68),(3.4,1.12,2.60),(3.5,1.1,2.52),(3.6,1.08,2.45),(3.7,1.07,2.38),(3.8,1.06,2.31),(3.9,1.05,2.25),(4.0,1.03,2.19),(4.2,1.04,2.09),(4.4,1.03,2.01),(4.6,1.03,1.94),(4.8,1.01,1.90),(5.0,0.96,1.86)])
MaterialsDB['Pt']=np.array([(0.5,3.91,7.71),(0.55,4.58,7.14),(0.6,5.13,6.75),(0.65,5.52,6.66),(0.7,5.71,6.83),(0.75,5.57,7.02),(0.8,5.31,7.04),(0.85,5.05,6.98),(0.9,4.77,6.91),(0.95,4.5,6.77),(1.,4.25,6.62),(1.1,3.86,6.24),(1.2,3.55,5.92),(1.3,3.29,5.61),(1.4,3.1,5.32),(1.5,2.92,5.07),(1.6,2.76,4.84),(1.7,2.63,4.64),(1.8,2.51,4.43),(1.9,2.38,4.26),(2.,2.3,4.07),(2.1,2.23,3.92),(2.2,2.17,3.77),(2.3,2.1,3.67),(2.4,2.03,3.54),(2.5,1.96,3.42),(2.6,1.91,3.30),(2.7,1.87,3.20),(2.8,1.83,3.10),(2.9,1.79,3.01),(3.,1.75,2.92),(3.2,1.68,2.76),(3.4,1.63,2.62),(3.6,1.58,2.48),(3.8,1.53,2.37),(4.,1.49,2.25),(4.2,1.45,2.14),(4.4,1.43,2.04),(4.6,1.39,1.95),(4.8,1.38,1.85),(5.,1.36,1.76)])
MaterialsDB['Ti']=np.array([(0.6,4.71,3.77),(0.7,4.38,3.89),(0.8,4.04,3.82),(0.9,3.8,3.65),(1.,3.62,3.52),(1.1,3.47,3.40),(1.2,3.35,3.30),(1.3,3.28,3.25),(1.4,3.17,3.28),(1.5,2.98,3.32),(1.6,2.74,3.30),(1.7,2.54,3.23),(1.8,2.36,3.11),(1.9,2.22,2.99),(2.,2.11,2.88),(2.1,2.01,2.77),(2.2,1.92,2.67),(2.3,1.86,2.56),(2.4,1.81,2.47),(2.5,1.78,2.39),(2.6,1.75,2.34),(2.7,1.71,2.29),(2.8,1.68,2.25),(2.9,1.63,2.21),(3.,1.59,2.17),(3.1,1.55,2.15),(3.2,1.5,2.12),(3.3,1.44,2.09),(3.4,1.37,2.06),(3.5,1.3,2.01),(3.6,1.24,1.96),(3.7,1.17,1.90),(3.8,1.11,1.83),(3.85,1.08,1.78),(3.9,1.06,1.73),(4.,1.04,1.62),(4.2,1.05,1.45),(4.4,1.13,1.33),(4.6,1.17,1.29),(4.8,1.21,1.23),(5.,1.24,1.21),(5.2,1.27,1.20)])
MaterialsDB['Ge']=np.array([(0.6,4.104,6.58e-07),(0.7,4.18,1.27e-04),(0.8,4.275,5.67e-03),(0.9,4.285,7.45e-02),(1.,4.325,8.09e-02),(1.1,4.385,0.103),(1.2,4.42,0.123),(1.3,4.495,0.167),(1.4,4.56,0.190),(1.5,4.635,0.298),(1.6,4.763,0.345),(1.7,4.897,0.401),(1.8,5.067,0.500),(1.9,5.38,0.540),(2.,5.588,0.933),(2.1,5.748,1.634),(2.2,5.283,2.049),(2.3,5.062,2.318),(2.4,4.61,2.455),(2.5,4.34,2.384),(2.6,4.18,2.309),(2.7,4.082,2.240),(2.8,4.035,2.181),(2.9,4.037,2.140),(3.,4.082,2.145),(3.1,4.141,2.215),(3.2,4.157,2.340),(3.3,4.128,2.469),(3.4,4.07,2.579),(3.5,4.02,2.667),(3.6,3.985,2.759),(3.7,3.958,2.863),(3.8,3.936,2.986),(3.9,3.92,3.137),(4.,3.905,3.336),(4.1,3.869,3.614),(4.2,3.745,4.009),(4.3,3.338,4.507),(4.4,2.516,4.669),(4.5,1.953,4.297),(4.6,1.72,3.960),(4.7,1.586,3.709),(4.8,1.498,3.509),(4.9,1.435,3.342),(5.,1.394,3.197),(5.1,1.37,3.073),(5.2,1.364,2.973),(5.3,1.371,2.897),(5.4,1.383,2.854),(5.5,1.38,2.842),(5.6,1.36,2.846),(5.7,1.293,2.163),(5.8,1.209,2.873),(5.9,1.108,2.813),(6.,1.30,2.340)])
MaterialsDB['Si']=np.array([(0.6199,3.449,0),(0.8093,3.4784,0),(1.033,3.5193,0),(1.1,3.5341,0.000013),(1.2,3.568825,0.00018),(1.3,3.60355,0.00226),(1.4,3.638275,0.00775),(1.5,3.673,0.005),(1.6,3.714,0.008),(1.7,3.752,0.01),(1.8,3.796,0.013),(1.9,3.847,0.016),(2.,3.906,0.022),(2.1,3.969,0.030),(2.2,4.042,0.032),(2.3,4.123,0.048),(2.4,4.215,0.060),(2.5,4.32,0.073),(2.6,4.442,0.090),(2.7,4.583,0.130),(2.8,4.753,0.163),(2.9,4.961,0.203),(3.,5.222,0.269),(3.1,5.57,0.387),(3.2,6.062,0.630),(3.3,6.709,1.321),(3.4,6.522,2.705),(3.5,5.61,3.014),(3.6,5.296,2.987),(3.7,5.156,3.058),(3.8,5.065,3.182),(3.9,5.016,3.346),(4.,5.01,3.587),(4.1,5.02,3.979),(4.2,4.888,4.639),(4.3,4.086,5.395),(4.4,3.12,5.344),(4.5,2.451,5.082),(4.6,1.988,4.678),(4.7,1.764,4.278),(4.8,1.658,3.979),(4.9,1.597,3.749),(5.,1.57,3.565),(5.1,1.571,3.429),(5.2,1.589,3.354),(5.3,1.579,3.353),(5.4,1.471,3.366),(5.5,1.34,3.302),(5.6,1.247,3.206),(5.7,1.18,3.112),(5.8,1.133,3.045),(5.9,1.083,2.982),(6.,1.01,2.909)])
MaterialsDB['aSi']=np.array([(0.6,3.44,0.0),(0.7,3.45,0.0),(0.8,3.48,0.0),(0.9,3.5,0.0),(1.0,3.54,0.0),(1.1,3.57,0.0),(1.2,3.61,0.0),(1.3,3.68,0.0),(1.4,3.77,0.0401),(1.5,3.86,0.0812),(1.6,3.93,0.136),(1.7,4.01,0.199),(1.8,4.09,0.271),(1.9,4.17,0.363),(2.0,4.23,0.461),(2.2,4.36,0.69),(2.4,4.46,0.969),(2.5,4.47,1.12),(2.6,4.49,1.28),(2.8,4.47,1.64),(3.0,4.38,2.02),(3.2,4.17,2.38),(3.4,3.9,2.66),(3.5,3.73,2.79),(3.6,3.55,2.88),(3.8,3.21,3.0),(4.0,2.87,3.06),(4.2,2.56,3.04),(4.4,2.3,2.99),(4.6,2.07,2.93),(4.8,1.86,2.85),(5.0,1.69,2.76),(5.5,1.35,2.51),(6.0,1.11,2.28),(6.5,0.951,2.07),(7.0,0.832,1.89),(7.5,0.735,1.74),(8.0,0.66,1.6),(8.5,0.597,1.47),(9.0,0.543,1.35),(9.5,0.497,1.24),(10.0,0.459,1.14),(10.5,0.423,1.04),(11.0,0.392,0.946),(11.5,0.363,0.847),(12.0,0.327,0.726)])

### Fitting at 250~700nm

In [ ]:
findFit('Ag', [250, 700], [2, 4, 4], [9, 5, 4])

In [ ]:
findFit('Au', [250, 700], [2, 4, 4, 4], [9, 5, 3, 2])

In [ ]:
findFit('Cu', [250, 700], [2, 4, 4, 4], [9, 5, 3, 2])

In [ ]:
findFit('Al', [250, 700], [2, 4], [9, 2])

In [ ]:
findFit('Cr', [250, 700], [2, 3, 4, 4], [9, 3, 2, 1])

In [ ]:
findFit('Ni', [250, 700], [2, 3, 4], [9, 4, 4])

In [ ]:
findFit('Ti', [250, 700], [2, 4, 4, 4], [9, 4, 3, 2])

In [ ]:
findFit('Pd', [250, 700], [2, 4, 4, 4], [9, 4, 3, 2])

In [ ]:
findFit('Pt', [250, 700], [2, 4, 4], [9, 4, 3])

In [ ]:
findFit('Ge', [250, 700], [4, 4, 4, 4])

In [ ]:
findFit('Si', [250, 700], [4, 4, 4, 4])

In [ ]:
findFit('aSi', [250, 700], [4, 4], [4])

### Fitting at 400~2000nm

In [ ]:
findFit('AgJC', [400, 2000], [2])

In [ ]:
findFit('Au', [400, 2000], [2, 4, 4], [9, 2])

In [ ]:
findFit('Cu', [400, 2000], [2, 4, 4], [9, 2])

In [ ]:
findFit('Al', [400, 2000], [2, 4, 4], [9])

In [ ]:
findFit('Cr', [400, 2000], [2, 4, 4, 4])

In [ ]:
findFit('Ni', [400, 2000], [2, 4, 4])

In [ ]:
findFit('Ti', [400, 1700], [2, 4, 4, 4], [9, 2.5, 1.5, 0.5])

In [ ]:
findFit('Pd', [400, 2000], [2, 4, 4])

In [ ]:
findFit('Pt', [400, 2200], [2, 4, 4, 4], [9])

In [ ]:
findFit('Ge', [400, 2000], [4, 4, 4])

In [ ]:
findFit('Si', [400, 2000], [4])

In [ ]:
findFit('aSi', [400, 2000], [4, 4], [4])

### All fitting results

In [32]:
print('250-700nm')
findFit('Ag', [250, 700], [2, 4, 4], [9, 5, 4], plot=False)
findFit('Au', [250, 700], [2, 4, 4, 4], [9, 5, 3, 2], plot=False)
findFit('Cu', [250, 700], [2, 4, 4, 4], [9, 5, 3, 2], plot=False)
findFit('Al', [250, 700], [2, 4], [9, 2], plot=False)
findFit('Cr', [250, 700], [2, 3, 4, 4], [9, 3, 2, 1], plot=False)
findFit('Ni', [250, 700], [2, 3, 4], [9, 4, 4], plot=False)
findFit('Ti', [250, 700], [2, 4, 4, 4], [9, 4, 3, 2], plot=False)
findFit('Pd', [250, 700], [2, 4, 4, 4], [9, 4, 3, 2], plot=False)
findFit('Pt', [250, 700], [2, 4, 4], [9, 4, 3], plot=False)
findFit('Ge', [250, 700], [4, 4, 4, 4], plot=False)
findFit('Si', [250, 700], [4, 4, 4, 4], plot=False)
findFit('aSi',[250, 700], [4, 4], plot=False)
print('400-2000nm')
findFit('AgJC', [400, 2000], [2], plot=False)
findFit('Au', [400, 2000], [2, 4, 4], [9, 2], plot=False)
findFit('Cu', [400, 2000], [2, 4, 4], [9, 2], plot=False)
findFit('Al', [400, 2000], [2, 4, 4], [9], plot=False)
findFit('Cr', [400, 2000], [2, 4, 4, 4], plot=False)
findFit('Ni', [400, 2000], [2, 4, 4], plot=False)
findFit('Ti', [400, 1700], [2, 4, 4, 4], [9, 2.5, 1.5, 0.5], plot=False)
findFit('Pd', [400, 2000], [2, 4, 4], plot=False)
findFit('Pt', [400, 2200], [2, 4, 4, 4], [9], plot=False)
findFit('Ge', [400, 2000], [4, 4, 4], plot=False)
findFit('Si', [400, 2000], [4, 4], plot=False)
findFit('aSi',[400, 2000], [4, 4], plot=False)
print('Done')

250-700nm
matter Ag = {{1.648},{{9.3602,0.1229},{5.4735,2.8361,1.8391,0.2496},{3.9866,0.3207,-0.0777,0.2551}}};
matter Au = {{2.2751},{{8.5081,0.0243},{3.4876,0.4267,-0.2116,0.0959},{3.0803,1.0339,1.8773,1.3745},{2.6144,0.174,0.0552,0.1018}}};
matter Cu = {{1.3889},{{8.814,0.0892},{4.7281,0.6755,0.1687,0.1876},{3.1693,2.6385,3.5153,0.6708},{2.1608,0.2616,-0.013,0.5105}}};
matter Al = {{1.0},{{15.2249,0.6945},{1.6816,0.3206,1.3603,-0.4233}}};
matter Cr = {{1.0},{{14.3486,4.4186},{3.2238,0.7842,2.8279},{2.755,0.7266,-2.2532,-5.3117},{1.5202,3.2276,4.6966,12.9959}}};
matter Ni = {{1.2656},{{10.2087,1.6734},{3.2808,1.4498,1.7095},{4.1563,0.9434,0.4141,0.8349}}};
matter Ti = {{1.0},{{8.8868,3.2677},{3.7903,1.2765,-10.4554,5.6631},{3.6757,1.271,11.4955,-4.7483},{1.4182,0.4732,2.1148,0.055}}};
matter Pd = {{13.3093},{{8.541,0.4616},{12.5662,6.1972,-2.059,-7.046},{1.1808,1.2419,16.3193,1.2766},{10.2029,7.7124,0.7661,-1.3891}}};
matter Pt = {{1.7368},{{10.4927,1.3232},{0.4154,7.934,-2.3308,3.34